<a href="https://colab.research.google.com/github/Kaia-nyoung/2025-ML-class/blob/main/9%EC%A3%BC%EC%B0%A8/UCI_HAR_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [12]:
def load_file(filepath):
    data = np.loadtxt(filepath)
    return data

In [11]:
def load_group(filenames, prefix=''):
    loaded = []
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    loaded = np.dstack(loaded)  # (샘플 수, 타임스텝, feature 수)
    return loaded

In [10]:
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    filenames = [
        'body_acc_x_'  + group + '.txt',
        'body_acc_y_'  + group + '.txt',
        'body_acc_z_'  + group + '.txt',
        'body_gyro_x_' + group + '.txt',
        'body_gyro_y_' + group + '.txt',
        'body_gyro_z_' + group + '.txt',
        'total_acc_x_' + group + '.txt',
        'total_acc_y_' + group + '.txt',
        'total_acc_z_' + group + '.txt'
    ]
    X = load_group(filenames, filepath)
    y = load_file(prefix + group + '/y_' + group + '.txt')
    return X, y

In [8]:
def load_dataset(dataset_root='/content/drive/MyDrive/ML-MK/UCI_HAR/'):
    trainX, trainy = load_dataset_group('train', dataset_root)
    testX,  testy  = load_dataset_group('test',  dataset_root)

    # 라벨이 1~6 → 0~5로 맞추고 원핫
    trainy_one_hot = to_categorical(trainy - 1)
    testy_one_hot  = to_categorical(testy - 1)
    return trainX, trainy, trainy_one_hot, testX, testy, testy_one_hot

In [15]:
trainX, trainy, trainy_one_hot, testX, testy, testy_one_hot = load_dataset()

X_train, X_val, y_train_one_hot, y_val_one_hot, y_train, y_val = train_test_split(
    trainX, trainy_one_hot, trainy,
    test_size=0.2,
    random_state=100
)

n_timesteps, n_features = X_train.shape[1], X_train.shape[2]
n_outputs = y_train_one_hot.shape[1]

print("✅ Data loaded successfully")
print("train:", X_train.shape, y_train_one_hot.shape)
print("val  :", X_val.shape,   y_val_one_hot.shape)
print("test :", testX.shape,   testy_one_hot.shape)

✅ Data loaded successfully
train: (5881, 128, 9) (5881, 6)
val  : (1471, 128, 9) (1471, 6)
test : (2947, 128, 9) (2947, 6)


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

In [25]:
def build_better_cnn():
    model = Sequential()
    model.add(Conv1D(64, 3, activation='relu', input_shape=(n_timesteps, n_features)))
    model.add(BatchNormalization())
    model.add(Conv1D(64, 3, activation='relu'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(0.3))

    model.add(Conv1D(128, 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(0.3))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_outputs, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [26]:
cnn_model = build_cnn_model()
history_cnn = cnn_model.fit(
    X_train, y_train_one_hot,
    validation_data=(X_val, y_val_one_hot),
    epochs=50,
    batch_size=64,
    verbose=1
)

Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


92/92 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.6405 - loss: 0.8858 - val_accuracy: 0.9048 - val_loss: 0.2918
Epoch 2/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8999 - loss: 0.2586 - val_accuracy: 0.9225 - val_loss: 0.1883
Epoch 3/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9509 - loss: 0.1296 - val_accuracy: 0.9232 - val_loss: 0.1603
Epoch 4/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9563 - loss: 0.1146 - val_accuracy: 0.9409 - val_loss: 0.1461
Epoch 5/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9605 - loss: 0.1069 - val_accuracy: 0.9409 - val_loss: 0.1389
Epoch 6/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9509 - loss: 0.1243 - val_accuracy: 0.9504 - val_loss: 0.1204
Epoch 7/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9580 - loss: 0.0994 - val_accuracy: 0.9341 - val_loss: 0.1339
Epoch 8/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9545 - loss: 0.0965 - val_accuracy: 0.9429 - val_loss: 0.1273
Ep

In [27]:
test_pred_proba = cnn_model.predict(testX)
test_pred = np.argmax(test_pred_proba, axis=-1)
test_true = testy.astype(int) - 1

93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [28]:
from sklearn.metrics import accuracy_score, classification_report
print("CNN Test Acc:", accuracy_score(test_true, test_pred))
print(classification_report(test_true, test_pred))

CNN Test Acc: 0.9151679674244995
              precision    recall  f1-score   support

           0       0.96      0.92      0.94       496
           1       0.92      0.91      0.91       471
           2       0.86      0.97      0.91       420
           3       0.91      0.77      0.83       491
           4       0.84      0.93      0.88       532
           5       0.99      1.00      1.00       537

    accuracy                           0.92      2947
   macro avg       0.92      0.91      0.91      2947
weighted avg       0.92      0.92      0.91      2947

